In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [3]:
# Constants
BATCH_SIZE = 32
IMG_SIZE = 256
CHANNELS = 3
EPOCHS = 50
NUM_CLASSES = 4

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Wheat_Dataset/Training/Wheat_Disease",
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

Found 3756 files belonging to 4 classes.


In [5]:
class_names = dataset.class_names
print("Classes:", class_names)

Classes: ['Black Rust', 'Brown Rust', 'Healthy Wheat', 'Yellow Rust']


In [6]:
for images_batch,label_batch in dataset.take(1):
    print(images_batch.shape)
    print(label_batch.numpy())

(32, 256, 256, 3)
[1 2 1 3 1 1 1 3 3 2 1 3 2 2 1 1 1 3 1 2 0 1 1 2 3 3 1 1 3 0 1 0]


In [7]:
for images_batch, label_batch in dataset.take(1):
    print(images_batch.shape)
    print(label_batch.numpy())

    plt.figure(figsize=(12, 9))
    for i in range(min(12, images_batch.shape[0])):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images_batch[i].numpy().astype("uint8"))
        plt.title(dataset.class_names[label_batch[i]])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [8]:
# 80% ==> Training
# 20% ==> test
#   10% ==> Validation
#   10% ==> Test

In [9]:
train_size = 0.8
len(dataset)

118

In [10]:
118*32

3776

In [11]:
train_size = 0.8
len(dataset) * train_size

94.4

In [12]:
train_ds = dataset.take(94)
len(train_ds)

94

In [13]:
test_ds = dataset.skip(94)
len(test_ds)

24

In [14]:
val_size = 0.1
len(dataset) * val_size

11.8

In [15]:
val_ds = test_ds.take(11)
len(val_ds)

11

In [16]:
test_ds = test_ds.skip(11)
len(test_ds)

13

In [17]:
import tensorflow as tf

def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1,
                               shuffle=True, shuffle_size=10000):
    # Total number of samples in the dataset
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=42)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

In [18]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [19]:
len(train_ds)

94

In [20]:
len(val_ds)

11

In [21]:
len(test_ds)

13

In [22]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [23]:
for image_batch , label_batch in dataset.take(1):
    print(image_batch[0].numpy()/255)

[[[0.38431373 0.56078434 0.23529412]
  [0.3529412  0.5019608  0.20784314]
  [0.44705883 0.54509807 0.31764707]
  ...
  [0.27450982 0.38039216 0.3137255 ]
  [0.23137255 0.34509805 0.27450982]
  [0.20392157 0.3254902  0.25490198]]

 [[0.41960785 0.5921569  0.24313726]
  [0.42745098 0.5803922  0.2627451 ]
  [0.32941177 0.4509804  0.1882353 ]
  ...
  [0.2784314  0.38431373 0.31764707]
  [0.23921569 0.3529412  0.28235295]
  [0.2        0.32156864 0.2509804 ]]

 [[0.4627451  0.63529414 0.2509804 ]
  [0.42352942 0.5921569  0.23137255]
  [0.43529412 0.5882353  0.27058825]
  ...
  [0.27058825 0.3764706  0.3019608 ]
  [0.23921569 0.3529412  0.28235295]
  [0.2        0.3137255  0.24313726]]

 ...

 [[0.10196079 0.20392157 0.05098039]
  [0.17254902 0.27450982 0.12941177]
  [0.08627451 0.1882353  0.05098039]
  ...
  [0.8156863  0.8117647  0.6       ]
  [0.7764706  0.77254903 0.54901963]
  [0.93333334 0.93333334 0.69803923]]

 [[0.10980392 0.21960784 0.07058824]
  [0.20784314 0.31764707 0.16862746]


In [24]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1.0/255)
])

In [25]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),
])

In [26]:
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS)),  # ✅ Correct way to set input shape

    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(NUM_CLASSES, activation="softmax")
])

In [27]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,812 (718.02 KB)

 Trainable params: 183,812 (718.02 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
 history = model.fit(
    train_ds,
    batch_size = BATCH_SIZE,
    validation_data = val_ds,
    verbose = 1,
    epochs = EPOCHS
)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1016s 5s/step - accuracy: 0.3323 - loss: 1.3512 - val_accuracy: 0.3864 - val_loss: 1.3084
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 468s 5s/step - accuracy: 0.4115 - loss: 1.2362 - val_accuracy: 0.4602 - val_loss: 1.2751
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 435s 5s/step - accuracy: 0.4649 - loss: 1.1341 - val_accuracy: 0.5170 - val_loss: 1.0575
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 439s 5s/step - accuracy: 0.4973 - loss: 1.0719 - val_accuracy: 0.5511 - val_loss: 1.0370
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 438s 5s/step - accuracy: 0.5338 - loss: 1.0394 - val_accuracy: 0.5682 - val_loss: 1.0728
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 432s 5s/step - accuracy: 0.5812 - loss: 1.0299 - val_accuracy: 0.6080 - val_loss: 0.9490
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 435s 5s/step - accuracy: 0.6042 - loss: 0.9466 - val_accuracy: 0.6449 - val_loss: 0.8938
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 445s 5s/step - accuracy: 0.6425 - loss: 0.9048 - val_accuracy: 0.6619 - 